# Inspect the properties on Wikidata

In this notebook we will inspect the different properties of Wikidata and show which ones are interesting to study in a statistical analysis.
The most essential thing to focus on is which properties are present in the maximum number of instances. It is also important that the instances share most of the properties.

To do this, we set up a query that retrieves all the properties but first we exclude the properties including an identifier (because there is no interest to study and this reduces the number of responses). We also exclude properties with too few instances (relative to the number of instances in the query). It drastically decreases the execution time of the request.

All the queries are first written on the SQLite database to keep them in memory. (to create it see. https://github.com/Semantic-Data-for-Humanities/Economists_Jurists/blob/development/Notebooks/Merge/Database_SQlite.ipynb).

In [1]:
# Import libraries usefull

from SPARQLWrapper import SPARQLWrapper, SPARQLWrapper2, JSON, TURTLE, XML, RDFXML
import pprint
import csv
# from bs4 import BeautifulSoup

from collections import Counter
from operator import itemgetter
import pandas as pd

import sqlite3 as sql
import time

from importlib import reload
from shutil import copyfile

In [2]:
import sparql_functions as spqf # It's made-home fonctions created by Francesco Beretta
# so they must to be in the same folder as this file.

# Execution of the query

In [29]:
### It's define the database ligne to use
pk_query = 50

# Connexion to the database
original_db = 'data/sparql_queries.db'
conn = sql.connect(original_db)

c = conn.cursor()

### It runs the query on the SQLite database to get the row values 
c.execute('SELECT * FROM query WHERE pk_query = ?', [pk_query]) ### a list around argument is needed for a string longer then one
#c.execute('SELECT * FROM query WHERE pk_query = 10')

rc = c.fetchone()

# close connexion
conn.close()


In [30]:
print(rc[2] +  "\n-----\n" + rc[4] +  "\n-----\n" +   rc[7]+  "\n\n\n------------------\n" +  rc[5] + "\n\n\n------------------\n")

SELECT: This query gets all the properties of lawyers.
I only exclude the properties with less of 5000 effectives and all properties with IDs
-----
https://query.wikidata.org/sparql
-----
2021-05-26 15:36:46


------------------
 SELECT  ?prop ?propLabel ?direction ?eff
WHERE{ { 
        SELECT  ?prop ?propLabel ?direction ?eff
WHERE{
    {
  SELECT   ?prop ?direction  (COUNT(*) as ?eff) # ?pLabel
 WHERE
  { 
  hint:Query hint:optimizer "None" .
      { ?person  wdt:P106  wd:Q40348. # lawyer: 68033 people
       ?person ?p ?o.
       ?prop wikibase:directClaim ?p .
       BIND("out" AS ?direction)
      }
  UNION
       {
       ?person  wdt:P106  wd:Q40348.
       ?s ?p ?person.
       ?prop wikibase:directClaim ?p .
       BIND("in" AS ?direction)
      }
  }

GROUP BY ?prop ?direction # ?pLabel
               }
SERVICE wikibase:label {bd:serviceParam wikibase:language "en" }
}
    }
FILTER (!regex(str(?propLabel), "ID"))
FILTER((?eff) > 5000)
}
ORDER BY DESC(?eff)



---------------

In [31]:
### Execute the SPARQL query wrapped in the function in the library _sparql_functions.py_
# The first setting correspond to SPARQL Endpoint, the seconde to the query
q = spqf.get_json_sparql_result(rc[4],rc[5])

<class 'dict'>


In [6]:
### This fonction retrieves and shapes time

# definition
def timestamp_formatted_for_file_name():
    is_now = time.strftime('%Y%m%d_%H%M%S')
    return is_now

# execution
timestamp_formatted_for_file_name()

'20210527_105507'

In [9]:
# Define the file addresses, the existing one and new one
original_db = 'data/sparql_queries.db'

timestamped_db_copy = 'data/sparql_queries_' + timestamp_formatted_for_file_name() + '.sqlite'

In [10]:
## Documentation:
# https://docs.python.org/3/library/shutil.htmlcopied_db = copyfile(original_db, timestamped_db_copy)

copied_db = copyfile(original_db, timestamped_db_copy)
copied_db

'data/sparql_queries_20210527_092846.sqlite'

In [32]:
### store the answer of the SPARQL endpoint in the 'result'

conn = sql.connect(original_db)
c = conn.cursor()
values = (pk_query, str(q),timestamp_formatted_for_file_name())

# https://www.techonthenet.com/sqlite/functions/now.php
c.execute("INSERT INTO result (fk_query, result, timestmp) VALUES (?,?,?)", values)
# commit the insertion and close the database
# !! REQUIRE to be enabled to commit to the SQLite database !!
conn.commit()
conn.close()

# Inspect the queries

In [53]:
### Inspect the resultat after an insert

# Choose the row of the database to get
pk_result = ('15')

# connexion to the database
original_db = 'data/sparql_queries.db'
conn = sql.connect(original_db)

### execute the query on the SQLite database to retrieve the values of the row
c = conn.cursor()
c.execute('SELECT * FROM result WHERE pk_result = ?', (pk_result,))
result_q = c.fetchone()

# close connexion
conn.close()
# result_q[3]

In [54]:
### Transform string to dict
## Doc.:
# https://stackoverflow.com/questions/988228/convert-a-string-representation-of-a-dictionary-to-a-dictionary
import ast
d = ast.literal_eval(result_q[3])
type(d)

dict

In [60]:
##### Transform the result into a list with a fonction of the library #####

#### Result of the query
#r_query = [l for l in spqf.sparql_result_to_list(d)]
#print(len(r_query))
#r_query

## Economist

In this query, we exclude this properties with less of 5000 statistical headcounts.
In total, There is 29322 instances

``
 [['http://www.wikidata.org/entity/P50', 'author', 'in', '73811'],
 ['http://www.wikidata.org/entity/P106', 'occupation', 'out', '62139'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '35764'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '35027'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '31056'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '30315'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '28157'],
 ['http://www.wikidata.org/entity/P27',
  'country of citizenship',
  'out',
  '26878'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'out', '20239'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '19449'],
 ['http://www.wikidata.org/entity/P213', 'ISNI', 'out', '18092'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '17322'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '16033'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '14632'],
 ['http://www.wikidata.org/entity/P184', 'doctoral advisor', 'in', '14254'],
 ['http://www.wikidata.org/entity/P185', 'doctoral student', 'out', '14020'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '11346'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '10814'],
 ['http://www.wikidata.org/entity/P18', 'image', 'out', '8888'],
 ['http://www.wikidata.org/entity/P102',
  'member of political party',
  'out',
  '6914'],
 ['http://www.wikidata.org/entity/P20', 'place of death', 'out', '6532'],
 ['http://www.wikidata.org/entity/P1559',
  'name in native language',
  'out',
  '6214'],
 ['http://www.wikidata.org/entity/P463', 'member of', 'out', '5497'],
 ['http://www.wikidata.org/entity/P1343',
  'described by source',
  'out',
  '5068'],
 ['http://www.wikidata.org/entity/P373', 'Commons category', 'out', '5038']]
 ``

## Jurist

In this query, we exclude this properties with less of 5000 statistical headcount.
In total, There is 24886 instances

``
[['http://www.wikidata.org/entity/P106', 'occupation', 'out', '66309'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '32133'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '31910'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '30681'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '30082'],
 ['http://www.wikidata.org/entity/P27',
  'country of citizenship',
  'out',
  '23782'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '22391'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '22305'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '17719'],
 ['http://www.wikidata.org/entity/P213', 'ISNI', 'out', '15196'],
 ['http://www.wikidata.org/entity/P20', 'place of death', 'out', '13572'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '13214'],
 ['http://www.wikidata.org/entity/P50', 'author', 'in', '12467'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '12143'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '11893'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '9286'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'out', '9180'],
 ['http://www.wikidata.org/entity/P1343',
  'described by source',
  'out',
  '7096'],
 ['http://www.wikidata.org/entity/P18', 'image', 'out', '7015'],
 ['http://www.wikidata.org/entity/P102',
  'member of political party',
  'out',
  '5276'],
 ['http://www.wikidata.org/entity/P463', 'member of', 'out', '5026']]
``

## Lawyers

In this query, we exclude this properties with less of 5000 statistical headcount.
In total, There is 68033 instances.

``
[['http://www.wikidata.org/entity/P106', 'occupation', 'out', '164319'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '76518'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '75720'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '71859'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '70724'],
 ['http://www.wikidata.org/entity/P27',
  'country of citizenship',
  'out',
  '67490'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '65950'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '65654'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '54125'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '47410'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '41384'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '37816'],
 ['http://www.wikidata.org/entity/P102',
  'member of political party',
  'out',
  '33891'],
 ['http://www.wikidata.org/entity/P18', 'image', 'out', '29423'],
 ['http://www.wikidata.org/entity/P20', 'place of death', 'out', '28912'],
 ['http://www.wikidata.org/entity/P213', 'ISNI', 'out', '20163'],
 ['http://www.wikidata.org/entity/P1559',
  'name in native language',
  'out',
  '18462'],
 ['http://www.wikidata.org/entity/P937', 'work location', 'out', '18027'],
 ['http://www.wikidata.org/entity/P50', 'author', 'in', '17875'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '15763'],
 ['http://www.wikidata.org/entity/P373', 'Commons category', 'out', '14022'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'out', '9973'],
 ['http://www.wikidata.org/entity/P1343',
  'described by source',
  'out',
  '7873'],
 ['http://www.wikidata.org/entity/P463', 'member of', 'out', '7482'],
 ['http://www.wikidata.org/entity/P40', 'child', 'out', '7019'],
 ['http://www.wikidata.org/entity/P22', 'father', 'in', '6755'],
 ['http://www.wikidata.org/entity/P40', 'child', 'in', '6326'],
 ['http://www.wikidata.org/entity/P119', 'place of burial', 'out', '6306']]
``

## Judge
In this query, we exclude this properties with less of 5000 statistical headcount.
In total, There is 68033 instances.

``
[['http://www.wikidata.org/entity/P106', 'occupation', 'out', '72093'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '36328'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '35758'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '34241'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '32441'],
 ['http://www.wikidata.org/entity/P27',
  'country of citizenship',
  'out',
  '30301'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '28188'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '26270'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '23800'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '21890'],
 ['http://www.wikidata.org/entity/P50', 'author', 'in', '17875'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '16516'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '14778'],
 ['http://www.wikidata.org/entity/P20', 'place of death', 'out', '11525'],
 ['http://www.wikidata.org/entity/P18', 'image', 'out', '9916'],
 ['http://www.wikidata.org/entity/P102',
  'member of political party',
  'out',
  '9249'],
 ['http://www.wikidata.org/entity/P213', 'ISNI', 'out', '8822'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '7869'],
 ['http://www.wikidata.org/entity/P22', 'father', 'in', '6755'],
 ['http://www.wikidata.org/entity/P937', 'work location', 'out', '6681'],
 ['http://www.wikidata.org/entity/P40', 'child', 'in', '6326'],
 ['http://www.wikidata.org/entity/P1559',
  'name in native language',
  'out',
  '5684'],
 ['http://www.wikidata.org/entity/P40', 'child', 'out', '5659'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'out', '5079']]
``

## Attorney

In this query, we exclude this properties with less of 100 statistical headcount.
In total, There is 100 instances.

``
[['http://www.wikidata.org/entity/P50', 'author', 'in', '17875'],
 ['http://www.wikidata.org/entity/P22', 'father', 'in', '6755'],
 ['http://www.wikidata.org/entity/P40', 'child', 'in', '6326'],
 ['http://www.wikidata.org/entity/P921', 'main subject', 'in', '4936'],
 ['http://www.wikidata.org/entity/P138', 'named after', 'in', '4817'],
 ['http://www.wikidata.org/entity/P3373', 'sibling', 'in', '3857'],
 ['http://www.wikidata.org/entity/P26', 'spouse', 'in', '3460'],
 ['http://www.wikidata.org/entity/P5826', 'majority opinion by', 'in', '3185'],
 ['http://www.wikidata.org/entity/P1817', 'addressee', 'in', '2799'],
 ['http://www.wikidata.org/entity/P991', 'successful candidate', 'in', '2762'],
 ['http://www.wikidata.org/entity/P710', 'participant', 'in', '2467'],
 ['http://www.wikidata.org/entity/P170', 'creator', 'in', '2059'],
 ['http://www.wikidata.org/entity/P180', 'depicts', 'in', '1759'],
 ['http://www.wikidata.org/entity/P161', 'cast member', 'in', '1725'],
 ['http://www.wikidata.org/entity/P127', 'owned by', 'in', '1444'],
 ['http://www.wikidata.org/entity/P112', 'founded by', 'in', '1443'],
 ['http://www.wikidata.org/entity/P1038', 'relative', 'in', '1221'],
 ['http://www.wikidata.org/entity/P1594', 'judge', 'in', '1205'],
 ['http://www.wikidata.org/entity/P1889', 'different from', 'in', '1161'],
 ['http://www.wikidata.org/entity/P301',
  "category's main topic",
  'in',
  '1096'],
 ['http://www.wikidata.org/entity/P726', 'candidate', 'in', '1028'],
 ['http://www.wikidata.org/entity/P488', 'chairperson', 'in', '1015'],
 ['http://www.wikidata.org/entity/P527', 'has part', 'in', '982'],
 ['http://www.wikidata.org/entity/P58', 'screenwriter', 'in', '869'],
 ['http://www.wikidata.org/entity/P1891', 'signatory', 'in', '763'],
 ['http://www.wikidata.org/entity/P823', 'speaker', 'in', '752'],
 ['http://www.wikidata.org/entity/P1308', 'officeholder', 'in', '691'],
 ['http://www.wikidata.org/entity/P6', 'head of government', 'in', '655'],
 ['http://www.wikidata.org/entity/P5030', 'talk show guest', 'in', '645'],
 ['http://www.wikidata.org/entity/P175', 'performer', 'in', '522'],
 ['http://www.wikidata.org/entity/P371', 'presenter', 'in', '476'],
 ['http://www.wikidata.org/entity/P162', 'producer', 'in', '447'],
 ['http://www.wikidata.org/entity/P655', 'translator', 'in', '339'],
 ['http://www.wikidata.org/entity/P57', 'director', 'in', '331'],
 ['http://www.wikidata.org/entity/P3342', 'significant person', 'in', '328'],
 ['http://www.wikidata.org/entity/P106', 'occupation', 'out', '283'],
 ['http://www.wikidata.org/entity/P98', 'editor', 'in', '282'],
 ['http://www.wikidata.org/entity/P737', 'influenced by', 'in', '248'],
 ['http://www.wikidata.org/entity/P25', 'mother', 'in', '236'],
 ['http://www.wikidata.org/entity/P86', 'composer', 'in', '233'],
 ['http://www.wikidata.org/entity/P1346', 'winner', 'in', '222'],
 ['http://www.wikidata.org/entity/P547', 'commemorates', 'in', '220'],
 ['http://www.wikidata.org/entity/P971',
  'category combines topics',
  'in',
  '211'],
 ['http://www.wikidata.org/entity/P1598', 'consecrator', 'in', '208'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'in', '198'],
 ['http://www.wikidata.org/entity/P1037', 'director / manager', 'in', '191'],
 ['http://www.wikidata.org/entity/P676', 'lyrics by', 'in', '173'],
 ['http://www.wikidata.org/entity/P674', 'characters', 'in', '165'],
 ['http://www.wikidata.org/entity/P1593', 'defender', 'in', '162'],
 ['http://www.wikidata.org/entity/P184', 'doctoral advisor', 'in', '160'],
 ['http://www.wikidata.org/entity/P1855',
  'Wikidata property example',
  'in',
  '152'],
 ['http://www.wikidata.org/entity/P1066', 'student of', 'in', '142'],
 ['http://www.wikidata.org/entity/P1877', 'after a work by', 'in', '142'],
 ['http://www.wikidata.org/entity/P802', 'student', 'in', '137'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '129'],
 ['http://www.wikidata.org/entity/P185', 'doctoral student', 'in', '128'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '125'],
 ['http://www.wikidata.org/entity/P451', 'unmarried partner', 'in', '124'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '124'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '124'],
 ['http://www.wikidata.org/entity/P27',
  'country of citizenship',
  'out',
  '114'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '111'],
 ['http://www.wikidata.org/entity/P88', 'commissioned by', 'in', '107'],
 ['http://www.wikidata.org/entity/P542', 'officially opened by', 'in', '105']]
``

## Advocate

In this query, we exclude this properties with less of 100 statistical headcount.
In total, There is 702 instances.

``
[['http://www.wikidata.org/entity/P50', 'author', 'in', '17875'],
 ['http://www.wikidata.org/entity/P22', 'father', 'in', '6755'],
 ['http://www.wikidata.org/entity/P40', 'child', 'in', '6326'],
 ['http://www.wikidata.org/entity/P921', 'main subject', 'in', '4937'],
 ['http://www.wikidata.org/entity/P138', 'named after', 'in', '4817'],
 ['http://www.wikidata.org/entity/P3373', 'sibling', 'in', '3857'],
 ['http://www.wikidata.org/entity/P26', 'spouse', 'in', '3460'],
 ['http://www.wikidata.org/entity/P5826', 'majority opinion by', 'in', '3185'],
 ['http://www.wikidata.org/entity/P1817', 'addressee', 'in', '2799'],
 ['http://www.wikidata.org/entity/P991', 'successful candidate', 'in', '2762'],
 ['http://www.wikidata.org/entity/P710', 'participant', 'in', '2467'],
 ['http://www.wikidata.org/entity/P106', 'occupation', 'out', '2172'],
 ['http://www.wikidata.org/entity/P170', 'creator', 'in', '2059'],
 ['http://www.wikidata.org/entity/P180', 'depicts', 'in', '1759'],
 ['http://www.wikidata.org/entity/P161', 'cast member', 'in', '1725'],
 ['http://www.wikidata.org/entity/P127', 'owned by', 'in', '1444'],
 ['http://www.wikidata.org/entity/P112', 'founded by', 'in', '1443'],
 ['http://www.wikidata.org/entity/P1038', 'relative', 'in', '1221'],
 ['http://www.wikidata.org/entity/P1594', 'judge', 'in', '1205'],
 ['http://www.wikidata.org/entity/P1889', 'different from', 'in', '1161'],
 ['http://www.wikidata.org/entity/P301',
  "category's main topic",
  'in',
  '1096'],
 ['http://www.wikidata.org/entity/P726', 'candidate', 'in', '1028'],
 ['http://www.wikidata.org/entity/P488', 'chairperson', 'in', '1015'],
 ['http://www.wikidata.org/entity/P527', 'has part', 'in', '982'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '881'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '874'],
 ['http://www.wikidata.org/entity/P58', 'screenwriter', 'in', '869'],
 ['http://www.wikidata.org/entity/P27',
  'country of citizenship',
  'out',
  '812'],
 ['http://www.wikidata.org/entity/P1891', 'signatory', 'in', '763'],
 ['http://www.wikidata.org/entity/P823', 'speaker', 'in', '752'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '746'],
 ['http://www.wikidata.org/entity/P1308', 'officeholder', 'in', '691'],
 ['http://www.wikidata.org/entity/P6', 'head of government', 'in', '655'],
 ['http://www.wikidata.org/entity/P5030', 'talk show guest', 'in', '645'],
 ['http://www.wikidata.org/entity/P69', 'educated at', 'out', '604'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '525'],
 ['http://www.wikidata.org/entity/P175', 'performer', 'in', '522'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '514'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '503'],
 ['http://www.wikidata.org/entity/P371', 'presenter', 'in', '476'],
 ['http://www.wikidata.org/entity/P162', 'producer', 'in', '447'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '433'],
 ['http://www.wikidata.org/entity/P18', 'image', 'out', '381'],
 ['http://www.wikidata.org/entity/P655', 'translator', 'in', '339'],
 ['http://www.wikidata.org/entity/P57', 'director', 'in', '331'],
 ['http://www.wikidata.org/entity/P3342', 'significant person', 'in', '328'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '321'],
 ['http://www.wikidata.org/entity/P98', 'editor', 'in', '282'],
 ['http://www.wikidata.org/entity/P102',
  'member of political party',
  'out',
  '270'],
 ['http://www.wikidata.org/entity/P737', 'influenced by', 'in', '248'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '248'],
 ['http://www.wikidata.org/entity/P25', 'mother', 'in', '236'],
 ['http://www.wikidata.org/entity/P86', 'composer', 'in', '233'],
 ['http://www.wikidata.org/entity/P1346', 'winner', 'in', '222'],
 ['http://www.wikidata.org/entity/P547', 'commemorates', 'in', '220'],
 ['http://www.wikidata.org/entity/P971',
  'category combines topics',
  'in',
  '211'],
 ['http://www.wikidata.org/entity/P1598', 'consecrator', 'in', '208'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'in', '198'],
 ['http://www.wikidata.org/entity/P1037', 'director / manager', 'in', '191'],
 ['http://www.wikidata.org/entity/P213', 'ISNI', 'out', '183'],
 ['http://www.wikidata.org/entity/P676', 'lyrics by', 'in', '173'],
 ['http://www.wikidata.org/entity/P674', 'characters', 'in', '165'],
 ['http://www.wikidata.org/entity/P373', 'Commons category', 'out', '165'],
 ['http://www.wikidata.org/entity/P1593', 'defender', 'in', '162'],
 ['http://www.wikidata.org/entity/P184', 'doctoral advisor', 'in', '160'],
 ['http://www.wikidata.org/entity/P937', 'work location', 'out', '159'],
 ['http://www.wikidata.org/entity/P20', 'place of death', 'out', '155'],
 ['http://www.wikidata.org/entity/P1855',
  'Wikidata property example',
  'in',
  '152'],
 ['http://www.wikidata.org/entity/P1066', 'student of', 'in', '142'],
 ['http://www.wikidata.org/entity/P1877', 'after a work by', 'in', '142'],
 ['http://www.wikidata.org/entity/P802', 'student', 'in', '137'],
 ['http://www.wikidata.org/entity/P40', 'child', 'out', '137'],
 ['http://www.wikidata.org/entity/P1559',
  'name in native language',
  'out',
  '131'],
 ['http://www.wikidata.org/entity/P185', 'doctoral student', 'in', '128'],
 ['http://www.wikidata.org/entity/P451', 'unmarried partner', 'in', '124'],
 ['http://www.wikidata.org/entity/P88', 'commissioned by', 'in', '107'],
 ['http://www.wikidata.org/entity/P512', 'academic degree', 'out', '107'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '107'],
 ['http://www.wikidata.org/entity/P551', 'residence', 'out', '106'],
 ['http://www.wikidata.org/entity/P542', 'officially opened by', 'in', '105']]
``

## Magistrate

In this query, we exclude this properties with less of 50 statistical headcount.
In total, There is 77 instances.

``
[['http://www.wikidata.org/entity/P50', 'author', 'in', '17875'],
 ['http://www.wikidata.org/entity/P22', 'father', 'in', '6755'],
 ['http://www.wikidata.org/entity/P40', 'child', 'in', '6326'],
 ['http://www.wikidata.org/entity/P921', 'main subject', 'in', '4937'],
 ['http://www.wikidata.org/entity/P138', 'named after', 'in', '4817'],
 ['http://www.wikidata.org/entity/P3373', 'sibling', 'in', '3857'],
 ['http://www.wikidata.org/entity/P26', 'spouse', 'in', '3460'],
 ['http://www.wikidata.org/entity/P5826', 'majority opinion by', 'in', '3185'],
 ['http://www.wikidata.org/entity/P1817', 'addressee', 'in', '2799'],
 ['http://www.wikidata.org/entity/P991', 'successful candidate', 'in', '2762'],
 ['http://www.wikidata.org/entity/P710', 'participant', 'in', '2467'],
 ['http://www.wikidata.org/entity/P170', 'creator', 'in', '2059'],
 ['http://www.wikidata.org/entity/P180', 'depicts', 'in', '1759'],
 ['http://www.wikidata.org/entity/P161', 'cast member', 'in', '1725'],
 ['http://www.wikidata.org/entity/P127', 'owned by', 'in', '1444'],
 ['http://www.wikidata.org/entity/P112', 'founded by', 'in', '1443'],
 ['http://www.wikidata.org/entity/P1038', 'relative', 'in', '1221'],
 ['http://www.wikidata.org/entity/P1594', 'judge', 'in', '1205'],
 ['http://www.wikidata.org/entity/P1889', 'different from', 'in', '1161'],
 ['http://www.wikidata.org/entity/P301',
  "category's main topic",
  'in',
  '1096'],
 ['http://www.wikidata.org/entity/P726', 'candidate', 'in', '1028'],
 ['http://www.wikidata.org/entity/P488', 'chairperson', 'in', '1015'],
 ['http://www.wikidata.org/entity/P527', 'has part', 'in', '982'],
 ['http://www.wikidata.org/entity/P58', 'screenwriter', 'in', '869'],
 ['http://www.wikidata.org/entity/P1891', 'signatory', 'in', '763'],
 ['http://www.wikidata.org/entity/P823', 'speaker', 'in', '752'],
 ['http://www.wikidata.org/entity/P1308', 'officeholder', 'in', '691'],
 ['http://www.wikidata.org/entity/P6', 'head of government', 'in', '655'],
 ['http://www.wikidata.org/entity/P5030', 'talk show guest', 'in', '645'],
 ['http://www.wikidata.org/entity/P175', 'performer', 'in', '522'],
 ['http://www.wikidata.org/entity/P371', 'presenter', 'in', '476'],
 ['http://www.wikidata.org/entity/P162', 'producer', 'in', '447'],
 ['http://www.wikidata.org/entity/P655', 'translator', 'in', '339'],
 ['http://www.wikidata.org/entity/P57', 'director', 'in', '331'],
 ['http://www.wikidata.org/entity/P3342', 'significant person', 'in', '328'],
 ['http://www.wikidata.org/entity/P98', 'editor', 'in', '282'],
 ['http://www.wikidata.org/entity/P737', 'influenced by', 'in', '248'],
 ['http://www.wikidata.org/entity/P25', 'mother', 'in', '236'],
 ['http://www.wikidata.org/entity/P86', 'composer', 'in', '233'],
 ['http://www.wikidata.org/entity/P1346', 'winner', 'in', '222'],
 ['http://www.wikidata.org/entity/P547', 'commemorates', 'in', '220'],
 ['http://www.wikidata.org/entity/P971',
  'category combines topics',
  'in',
  '211'],
 ['http://www.wikidata.org/entity/P1598', 'consecrator', 'in', '208'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'in', '198'],
 ['http://www.wikidata.org/entity/P1037', 'director / manager', 'in', '191'],
 ['http://www.wikidata.org/entity/P676', 'lyrics by', 'in', '173'],
 ['http://www.wikidata.org/entity/P106', 'occupation', 'out', '171'],
 ['http://www.wikidata.org/entity/P674', 'characters', 'in', '165'],
 ['http://www.wikidata.org/entity/P1593', 'defender', 'in', '162'],
 ['http://www.wikidata.org/entity/P184', 'doctoral advisor', 'in', '160'],
 ['http://www.wikidata.org/entity/P1855',
  'Wikidata property example',
  'in',
  '152'],
 ['http://www.wikidata.org/entity/P1066', 'student of', 'in', '142'],
 ['http://www.wikidata.org/entity/P1877', 'after a work by', 'in', '142'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '141'],
 ['http://www.wikidata.org/entity/P802', 'student', 'in', '137'],
 ['http://www.wikidata.org/entity/P185', 'doctoral student', 'in', '128'],
 ['http://www.wikidata.org/entity/P735', 'given name', 'out', '125'],
 ['http://www.wikidata.org/entity/P451', 'unmarried partner', 'in', '124'],
 ['http://www.wikidata.org/entity/P31', 'instance of', 'out', '120'],
 ['http://www.wikidata.org/entity/P21', 'sex or gender', 'out', '117'],
 ['http://www.wikidata.org/entity/P88', 'commissioned by', 'in', '107'],
 ['http://www.wikidata.org/entity/P542', 'officially opened by', 'in', '105'],
 ['http://www.wikidata.org/entity/P166', 'award received', 'out', '101'],
 ['http://www.wikidata.org/entity/P6241', 'collection creator', 'in', '99'],
 ['http://www.wikidata.org/entity/P569', 'date of birth', 'out', '98'],
 ['http://www.wikidata.org/entity/P3975', 'secretary general', 'in', '96'],
 ['http://www.wikidata.org/entity/P1269', 'facet of', 'in', '96'],
 ['http://www.wikidata.org/entity/P1591', 'defendant', 'in', '92'],
 ['http://www.wikidata.org/entity/P3320', 'board member', 'in', '89'],
 ['http://www.wikidata.org/entity/P27', 'country of citizenship', 'out', '87'],
 ['http://www.wikidata.org/entity/P84', 'architect', 'in', '85'],
 ['http://www.wikidata.org/entity/P1412',
  'languages spoken, written or signed',
  'out',
  '84'],
 ['http://www.wikidata.org/entity/P35', 'head of state', 'in', '83'],
 ['http://www.wikidata.org/entity/P169',
  'chief executive officer',
  'in',
  '83'],
 ['http://www.wikidata.org/entity/P767',
  'contributor to the creative work or subject',
  'in',
  '79'],
 ['http://www.wikidata.org/entity/P19', 'place of birth', 'out', '75'],
 ['http://www.wikidata.org/entity/P825', 'dedicated to', 'in', '73'],
 ['http://www.wikidata.org/entity/P734', 'family name', 'out', '72'],
 ['http://www.wikidata.org/entity/P195', 'collection', 'in', '68'],
 ['http://www.wikidata.org/entity/P466', 'occupant', 'in', '68'],
 ['http://www.wikidata.org/entity/P344',
  'director of photography',
  'in',
  '66'],
 ['http://www.wikidata.org/entity/P87', 'librettist', 'in', '63'],
 ['http://www.wikidata.org/entity/P1040', 'film editor', 'in', '61'],
 ['http://www.wikidata.org/entity/P570', 'date of death', 'out', '53']]
``

## Advocate in France

In this query, we exclude this properties with less of 50 statistical headcount.
In total, There is 29 instances.

``
[['http://www.wikidata.org/entity/P50', 'author', 'in', '17875'],
 ['http://www.wikidata.org/entity/P22', 'father', 'in', '6755'],
 ['http://www.wikidata.org/entity/P40', 'child', 'in', '6326'],
 ['http://www.wikidata.org/entity/P921', 'main subject', 'in', '4937'],
 ['http://www.wikidata.org/entity/P138', 'named after', 'in', '4817'],
 ['http://www.wikidata.org/entity/P3373', 'sibling', 'in', '3857'],
 ['http://www.wikidata.org/entity/P26', 'spouse', 'in', '3460'],
 ['http://www.wikidata.org/entity/P5826', 'majority opinion by', 'in', '3185'],
 ['http://www.wikidata.org/entity/P1817', 'addressee', 'in', '2799'],
 ['http://www.wikidata.org/entity/P991', 'successful candidate', 'in', '2762'],
 ['http://www.wikidata.org/entity/P710', 'participant', 'in', '2467'],
 ['http://www.wikidata.org/entity/P170', 'creator', 'in', '2059'],
 ['http://www.wikidata.org/entity/P180', 'depicts', 'in', '1759'],
 ['http://www.wikidata.org/entity/P161', 'cast member', 'in', '1725'],
 ['http://www.wikidata.org/entity/P127', 'owned by', 'in', '1444'],
 ['http://www.wikidata.org/entity/P112', 'founded by', 'in', '1443'],
 ['http://www.wikidata.org/entity/P1038', 'relative', 'in', '1221'],
 ['http://www.wikidata.org/entity/P1594', 'judge', 'in', '1205'],
 ['http://www.wikidata.org/entity/P1889', 'different from', 'in', '1161'],
 ['http://www.wikidata.org/entity/P301',
  "category's main topic",
  'in',
  '1096'],
 ['http://www.wikidata.org/entity/P726', 'candidate', 'in', '1028'],
 ['http://www.wikidata.org/entity/P488', 'chairperson', 'in', '1015'],
 ['http://www.wikidata.org/entity/P527', 'has part', 'in', '982'],
 ['http://www.wikidata.org/entity/P58', 'screenwriter', 'in', '869'],
 ['http://www.wikidata.org/entity/P1891', 'signatory', 'in', '763'],
 ['http://www.wikidata.org/entity/P823', 'speaker', 'in', '752'],
 ['http://www.wikidata.org/entity/P1308', 'officeholder', 'in', '691'],
 ['http://www.wikidata.org/entity/P6', 'head of government', 'in', '655'],
 ['http://www.wikidata.org/entity/P5030', 'talk show guest', 'in', '645'],
 ['http://www.wikidata.org/entity/P175', 'performer', 'in', '522'],
 ['http://www.wikidata.org/entity/P371', 'presenter', 'in', '476'],
 ['http://www.wikidata.org/entity/P162', 'producer', 'in', '447'],
 ['http://www.wikidata.org/entity/P655', 'translator', 'in', '339'],
 ['http://www.wikidata.org/entity/P57', 'director', 'in', '331'],
 ['http://www.wikidata.org/entity/P3342', 'significant person', 'in', '328'],
 ['http://www.wikidata.org/entity/P98', 'editor', 'in', '282'],
 ['http://www.wikidata.org/entity/P737', 'influenced by', 'in', '248'],
 ['http://www.wikidata.org/entity/P25', 'mother', 'in', '236'],
 ['http://www.wikidata.org/entity/P86', 'composer', 'in', '233'],
 ['http://www.wikidata.org/entity/P1346', 'winner', 'in', '222'],
 ['http://www.wikidata.org/entity/P547', 'commemorates', 'in', '220'],
 ['http://www.wikidata.org/entity/P971',
  'category combines topics',
  'in',
  '211'],
 ['http://www.wikidata.org/entity/P1598', 'consecrator', 'in', '208'],
 ['http://www.wikidata.org/entity/P108', 'employer', 'in', '198'],
 ['http://www.wikidata.org/entity/P1037', 'director / manager', 'in', '191'],
 ['http://www.wikidata.org/entity/P676', 'lyrics by', 'in', '173'],
 ['http://www.wikidata.org/entity/P674', 'characters', 'in', '165'],
 ['http://www.wikidata.org/entity/P1593', 'defender', 'in', '162'],
 ['http://www.wikidata.org/entity/P184', 'doctoral advisor', 'in', '160'],
 ['http://www.wikidata.org/entity/P1855',
  'Wikidata property example',
  'in',
  '152'],
 ['http://www.wikidata.org/entity/P1066', 'student of', 'in', '142'],
 ['http://www.wikidata.org/entity/P1877', 'after a work by', 'in', '142'],
 ['http://www.wikidata.org/entity/P802', 'student', 'in', '137'],
 ['http://www.wikidata.org/entity/P185', 'doctoral student', 'in', '128'],
 ['http://www.wikidata.org/entity/P451', 'unmarried partner', 'in', '124'],
 ['http://www.wikidata.org/entity/P88', 'commissioned by', 'in', '107'],
 ['http://www.wikidata.org/entity/P542', 'officially opened by', 'in', '105'],
 ['http://www.wikidata.org/entity/P6241', 'collection creator', 'in', '99'],
 ['http://www.wikidata.org/entity/P3975', 'secretary general', 'in', '96'],
 ['http://www.wikidata.org/entity/P1269', 'facet of', 'in', '96'],
 ['http://www.wikidata.org/entity/P1591', 'defendant', 'in', '92'],
 ['http://www.wikidata.org/entity/P3320', 'board member', 'in', '89'],
 ['http://www.wikidata.org/entity/P84', 'architect', 'in', '85'],
 ['http://www.wikidata.org/entity/P35', 'head of state', 'in', '83'],
 ['http://www.wikidata.org/entity/P169',
  'chief executive officer',
  'in',
  '83'],
 ['http://www.wikidata.org/entity/P767',
  'contributor to the creative work or subject',
  'in',
  '79'],
 ['http://www.wikidata.org/entity/P825', 'dedicated to', 'in', '73'],
 ['http://www.wikidata.org/entity/P195', 'collection', 'in', '68'],
 ['http://www.wikidata.org/entity/P466', 'occupant', 'in', '68'],
 ['http://www.wikidata.org/entity/P106', 'occupation', 'out', '68'],
 ['http://www.wikidata.org/entity/P344',
  'director of photography',
  'in',
  '66'],
 ['http://www.wikidata.org/entity/P87', 'librettist', 'in', '63'],
 ['http://www.wikidata.org/entity/P1040', 'film editor', 'in', '61'],
 ['http://www.wikidata.org/entity/P39', 'position held', 'out', '56']]
``